In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import random
import matplotlib.pyplot as plt
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Conv2DTranspose, BatchNormalization, Activation, Dropout
from tensorflow.keras.optimizers import Adadelta, Nadam ,Adam,SGD
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.utils import  plot_model ,Sequence
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.preprocessing.image import load_img,img_to_array
import tensorflow as tf
from tensorflow.python.keras.losses import binary_crossentropy
from scipy.ndimage import morphology as mp

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
from glob import glob  # for getting list paths of image and labels
from random import choice,sample
from matplotlib import pyplot as plt
import cv2 # saving and loading images
from sklearn.model_selection import train_test_split
# Any results you write to the current directory are saved as output.

In [ ]:
import wandb
from wandb.keras import WandbCallback
wandb.login(key="19d16bb396bf34bc51e649940437bde54ec127ff")

In [ ]:
train_img_dir = '../input/multiclass-seg/multiclass_segmentation/cropped_train/instrument_dataset/images' 
train_mask_dir = '../input/multiclass-seg/multiclass_segmentation/cropped_train/instrument_dataset/instruments_masks'
train_imgs = os.listdir(train_img_dir)# if you have an error take a look here ...
train_masks = os.listdir(train_mask_dir)
train_imgs= sorted([ i for i in train_imgs ])
train_masks= sorted([ i for i in train_masks ])
print(len(train_imgs))
x_set = []
labels = []
for train_img in train_imgs :
    x_set.append(train_img)
    label = []
    for train_mask in train_masks:
        if((train_img[0:2] in train_mask[0:2] and train_img[-12:-4] in train_mask[-12:-4]) or (train_img[0] in train_mask[0]!='1' and train_img[0]!='2' and train_img[-12:-4] in train_mask[-12:-4])):
            label.append(train_mask)
        if (len(label) == 8):
            labels.append(label)
            break

In [ ]:
print(len(x_set))
print(len(labels))

In [ ]:
print(len(labels))
print(len(x_set))

In [ ]:
val_img_dir =  train_img_dir
val_mask_dir = train_mask_dir
val_size = 360
val_imgs = []
val_masks = []
train_imgs, val_imgs, train_masks, val_masks = train_test_split(x_set, labels, test_size=0.2, random_state=42)
print(len(val_imgs))
print(len(val_masks))

In [ ]:
print(val_masks[100])
print(val_imgs[100])

In [ ]:
from scipy import ndimage

In [ ]:
class DataGenerator(Sequence):
    'Generates data for Keras'
    
    def __init__(self, images,image_dir,labels,label_dir ,batch_size=16, dim=(224,224,3) ,shuffle=True):
        'Initialization'
        self.dim = dim
        self.images = images
        self.image_dir = image_dir
        self.labels = labels
        self.label_dir = label_dir
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.images) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [k for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.images))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        batch_imgs = list()
        batch_labels = list()

        # Generate data
        for i in list_IDs_temp:
#             degree=np.random.random() * 360
            # Store sample
            img = load_img(self.image_dir + '/' + self.images[i] ,target_size=self.dim)
            img = img_to_array(img)/255
#             img = ndimage.rotate(img, degree)
#             print(img)
            
           # Store class
            masks = []
            for m in range(8):
                label = load_img(self.label_dir + '/' + self.labels[i][m] ,target_size=self.dim)
                label = img_to_array(label)[:,:,0]
                label = label != 0
                label = mp.binary_erosion(mp.binary_erosion(label))
                label = mp.binary_dilation(mp.binary_dilation(mp.binary_dilation(label)))
                label = np.expand_dims((label)*1 , axis=2)
                #label = label/255
                masks.append(label)
                
            #####################
            batch_imgs.append(img)
            batch_labels.append(np.dstack((masks)))
        x = np.array(batch_imgs,dtype = np.float32 ) 
        y = np.array(batch_labels , dtype = np.float32 )
        #x = np.asarray(x).astype('float32').reshape((-1,1))
        #y = np.asarray(y).astype('float32').reshape((-1,1))
        return x,y

In [ ]:
dim=(128,128,3)
train_generator = DataGenerator(train_imgs,train_img_dir,train_masks,train_mask_dir,batch_size=32, dim=dim ,shuffle=True)
train_steps = train_generator.__len__()
train_steps

In [ ]:
X,y = train_generator.__getitem__(1)

In [ ]:
print(y.shape)

In [ ]:
t = 3
plt.subplot(121)
plt.subplot(122)
for i in range(8):
    plt.subplot(3,3,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(np.reshape(y[t,:,:,i],(128,128)))
plt.figure(figsize=(7,7))
plt.imshow(X[t])

In [ ]:
val_generator = DataGenerator(val_imgs,val_img_dir,val_masks,val_mask_dir,batch_size=16, dim=dim ,shuffle=True)
val_steps = val_generator.__len__()
val_steps

In [ ]:
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from pathlib import Path
import numpy as np
import os
from tensorflow.keras.layers import Conv2D,Dropout ,BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input,UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG16
import tensorflow as tf 
from sklearn.model_selection import train_test_split
import tensorflow.keras.initializers as initializers

In [ ]:
def conv_block(inputs,n_filters=32,drop_prob=0):
    #x = Dropout(0.9)(inputs)
    x = Conv2D(n_filters,3,padding="same")(inputs)
    #x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(n_filters,3,padding="same")(x)
    #x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(n_filters,3,padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    if(drop_prob > 0):
        x = Dropout(drop_prob)(x)
    return x
    #x = Conv2D(filterCount,3,padding="same")(x)
    #x = BatchNormalization()(x)
    #x = Activation("relu")(x)
    
    #return x

In [ ]:
def decoder_block(inputs,skip_features,n_filters=32):
    
    x = Conv2DTranspose(n_filters, 2, strides=2,activation='relu',padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x)
    
    #x = conv_block(x, filter_count)
    #x = conv_block(x, filter_count)
    #x = Dropout(0.5)(x)
    
    return x

In [ ]:
def unet_vgg16(input_shape,nb_classes=1,n_filters=32):
    inputs = Input(input_shape)
    vgg16 = VGG16(include_top=False,weights='imagenet',input_tensor = inputs)
    #vgg16.summary()
    #vgg16.trainable = True
    # the encoder 
    #falindices = [2,5,9,13,17]
    #trindices = [0,1,3,4,6,7,8,10,11,12,14,15,16,18]
    #for i in trindices :
        #vgg16.layers[i].trainable = False
    skip1 = vgg16.get_layer("block1_conv2").output
    skip2 = vgg16.get_layer("block2_conv2").output
    skip3 = vgg16.get_layer("block3_conv3").output
    skip4 = vgg16.get_layer("block4_conv3").output
    # the center
    center = vgg16.get_layer("block5_conv3").output

    # the decoder 
    #
    d1 = decoder_block(center,skip4,n_filters*16)
    d2 = decoder_block(d1,skip3,n_filters*8)
    d3 = decoder_block(d2,skip2,n_filters*4)
    d4 = decoder_block(d3,skip1,n_filters*2)
    #output
    conv1 = Conv2D(n_filters,3,padding="same",activation='relu')(d4)
    conv2 = Conv2D(n_filters,3,padding="same",activation='relu')(conv1)
    if (nb_classes == 1):
        outputs = Conv2D(1, (1,1), padding="same", activation="sigmoid")(conv1)
    else :
        outputs = Conv2D(nb_classes, 1,activation="sigmoid",padding="same")(conv2)
    model = Model(inputs, outputs, name="VGG16_U-Net")
    return model

In [ ]:
dim=(128,128,3)
input_shape = dim
model = unet_vgg16(input_shape,8)
model.summary()

In [ ]:
def jaccard_distance_loss(y_true, y_pred,smooth = 1):
    y_true = K.batch_flatten(y_true)
    y_pred = K.batch_flatten(y_pred)
    #y_true /= 255
    #y_pred /= 255
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return  jac * smooth

def jaccard_distance_loss_multi(y_true, y_pred):
    return 1 - IOU(y_true,y_pred)

def IOU(y_true, y_pred,smooth = 1):
    jac = 0
    for index in range(8):
        y_true_f = K.flatten(y_true[:,:,:,index])
        y_pred_f = K.flatten(y_pred[:,:,:,index])
        #y_true_f /= 255
        #y_pred_f /= 255
        #tf.print(y_pred_f - y_true_f)
        intersection = K.sum(K.abs(y_true_f * y_pred_f), axis=-1)
        sum_ = K.sum(K.abs(y_true_f) + K.abs(y_pred_f), axis=-1)
        jac += (intersection + smooth) / (sum_ - intersection + smooth)
    return  jac * smooth / 8

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())
def dice_coef_multilabel(y_true, y_pred):
    dice=0
    #print(y_pred.shape)
    for index in range(8):
        dice += dice_coef(y_true[:,:,:,index], y_pred[:,:,:,index])
    return dice/8 # taking average
def loss(y_true,y_pred):
    return 1 - dice_coef_multilabel(y_true,y_pred)

In [ ]:
print(IOU(y,y))

In [ ]:
wandb.init(project="surgical_image_segmenatation",entity="medait",name="multiclass-unet-vgg16")

In [ ]:
adam = Adam(learning_rate=0.000001)
model.compile(optimizer=adam, loss=jaccard_distance_loss_multi ,metrics = [IOU,dice_coef_multilabel, 'accuracy'])
mc = ModelCheckpoint(mode='max', filepath='top-weights.h5', monitor='val_dice_coef',save_best_only='True', save_weights_only='True', verbose=1)
es = EarlyStopping(mode='max', monitor='val_dice_coef', patience=3, verbose=1)
#ls = LearningRateScheduler(scheduler)
callbacks = [WandbCallback()]
#
model.metrics_names

In [ ]:
results = model.fit(train_generator, steps_per_epoch=train_steps,epochs=20,callbacks=callbacks,validation_data=val_generator,validation_steps=val_steps)

In [ ]:
model.load_weights('../input/data123/multiclass_unetvgg16.h5')

In [ ]:
def make_prediction(model,image,shape):
    img = img_to_array(load_img(image,target_size=shape))
    img = np.expand_dims(img,axis=0)/255.
    mask = model.predict(img)
    print(mask.shape)
    #
#     print(np.unique(mask,return_counts=True))
    masks = np.zeros((128,128,3))
#     for index in range(7):
#         mask[:,:,:,index] = (mask[0,:,:,index] > 0.5)*1
#         masks[:,:,index] = np.reshape(mask[:,:,:,index],(224,224))
    def rgb(*args) : return args
    cc = [
      rgb(243, 166, 131),
     rgb(207, 106, 135),
     rgb(225, 95, 65),
        rgb(119, 139, 235),
     rgb(99, 205, 218),
        rgb(87, 75, 144),
     rgb(247, 143, 179)]
    for index,c in enumerate(cc):
#         if index == 2:break
        print(c)
        masks[mask[0,:,:,index] > 0.5] = c
    return masks

In [ ]:
image = "../input/multiclass-seg/multiclass_segmentation/cropped_train/instrument_dataset/images/17frame100.jpg"
img = img_to_array(load_img(image))
plt.imshow(img/255.)
img.shape

In [ ]:
image1 = "../input/multiclass-seg/multiclass_segmentation/cropped_train/instrument_dataset/instruments_masks/16backgroundframe209.png"
img = img_to_array(load_img(image1))
plt.imshow(img/255.)
img.shape

In [ ]:
mask = make_prediction(model,image,dim)
#mask2 = cv2.merge([mask,mask,mask]).astype('float32')
#print(img.shape,mask2.shape)
#mask2 = cv2.resize(mask2,(img.shape[1],img.shape[0]))
print(mask.shape)
plt.imshow(mask/255)
cv2.imwrite('multi_17100.png',mask)

In [ ]:
c = [(119, 139, 235),
      (243, 166, 131),
    (207, 106, 135),
     (225, 95, 65),
     (99, 205, 218),
     (87, 75, 144),
     (247, 143, 179)]
merged_mask = np.zeros((128,128))
for i in range(7):
    mask[:,:,i][mask[:,:,i]>0] = c[i]
    merged_mask += mask[:,:,i]

In [ ]:
def make_prediction(model,image,shape):
    img = img_to_array(load_img(image,target_size=shape))
    img = np.expand_dims(img,axis=0)/255.
    mask = model.predict(img)
    print(mask.shape)
    #
#     print(np.unique(mask,return_counts=True))
    masks = np.zeros((224,224,3))
#     for index in range(7):
#         mask[:,:,:,index] = (mask[0,:,:,index] > 0.5)*1
#         masks[:,:,index] = np.reshape(mask[:,:,:,index],(224,224))
    cc = [rgb(119, 139, 235),
      rgb(243, 166, 131),
     rgb(207, 106, 135),
     rgb(225, 95, 65),
     rgb(99, 205, 218),
     rgb(87, 75, 144),
     rgb(247, 143, 179)]
    for index,c in enumerate(cc):
#         if index == 2:break
        print(c)
        masks[mask[0,:,:,index] > 0.5] = c
    return masks

In [ ]:
plt.imshow(merged_mask)

In [ ]:
model.save_weights('multiclass_unet+vgg16')